In [207]:
import pandas as pd
import numpy as np

import requests
import bs4 as bs
from urllib.parse import urljoin
import pickle

import os

%matplotlib inline

# Test (Spain)

https://en.wikipedia.org/wiki/Category:Opinion_polling_in_Spain

In [208]:
#Get all election links

#restrictions in link
restrictions = ["Opinion polling","general election"]
#main url
url = "https://en.wikipedia.org/wiki/Category:Opinion_polling_in_Spain"

#Extract all links
links = []
r = requests.get(url)
if r.ok:
    html = bs.BeautifulSoup(r.text,"html.parser")
    for link in html.find_all("a"):
        if link.has_attr('href') and all([restr in link.text for restr in restrictions]): #check restrictions
            links.append(urljoin(url,link["href"]))
print(links)
                    
                    

['https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1986', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1989', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1993', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1996', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2000', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2004', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2008', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2011', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2015', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2016', 'https://en.wikipedia.org/wiki/

In [209]:
def check_not_number(x):
    try:
        float(x.split("/")[0])
        return False
    except:
        return True

In [220]:
all_tables = []
country = "Spain"
!mkdir data/individual_tables/$country

#position table in website
table_position = 0
for p_url in links:
    #Main ulr
    #p_url = "https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982"

    #Get columns
    r = requests.get(p_url)
    html = bs.BeautifulSoup(r.text,"html.parser")
    tables = html.find_all("table")[table_position]

    rows = tables.find_all("th")
    columns = [r.find("img")["alt"] if (r.find("img") is not None) else (r.find("a")["href"] if (r.find("a") is not None) else None) for r in rows ]
    columns = [_ for _ in columns if _ is not None]
    print(p_url)
    print(columns)
    #Get tables of polls
    polls = pd.read_html(r.text,header=0)[table_position]


    #Rename columns
    types = ["firm","date","error","size","lead","turnout"]
    new_cols = []
    for col in polls.columns:
        flag = 0
        for t in types:
            if t in col.lower():
                new_cols.append(t)
                flag = 1
                break
        if flag == 0:
            new_cols.append(columns.pop(0))

    polls.columns = new_cols
    print(list(polls.columns))
    polls["date"] = polls["date"].str.split("–").str[-1]
    polls = polls.loc[polls["date"].apply(check_not_number)]
    
    polls.to_csv("data/individual_tables/{}/{}.csv".format(country,p_url.split("/")[-1]))
    
    #all_tables.append(polls)

mkdir: cannot create directory ‘data/individual_tables/Spain’: File exists
https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982
['UCD', 'PSOE', 'PCE', 'AP–PDP', 'CiU', 'PNV', 'CDS']
['firm', 'date', 'UCD', 'PSOE', 'PCE', 'AP–PDP', 'CiU', 'PNV', 'CDS', 'error', 'size', 'lead']
https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1986
['PSOE', 'AP-PDP-PL', 'IU', 'CiU', 'CDS', 'PNV', 'PRD']
['firm', 'date', 'PSOE', 'AP-PDP-PL', 'IU', 'CiU', 'CDS', 'PNV', 'PRD', 'error', 'size', 'lead']
https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1989
['PSOE', 'AP', 'CDS', 'CiU', 'IU', 'PNV', 'HB', 'EE', 'PA', 'ERC', '/wiki/Galician_Coalition', 'PAR', '/wiki/Canarian_Independent_Groups', 'UV', '/wiki/Galician_Socialist_Party%E2%80%93Galician_Left', '/wiki/Confederation_of_the_Greens', 'PDP', 'EA', 'PP', '/wiki/Ruiz-Mateos_Group']
['firm', 'date', 'size', 'turnout', 'PSOE', 'AP', 'CDS', 'CiU', 'IU', 'PNV', 'HB', '

In [221]:
country = "Spain"

#combine tables
all_tables = []
for table in os.listdir("./data/individual_tables/{}".format(country)):
    all_tables.append(pd.read_csv("data/individual_tables/{}/{}".format(country,table),parse_dates=["date"],na_values=["–","?"]))
    print(all_tables[-1]["date"].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]


In [223]:
all_tables = pd.concat(all_tables)
all_tables.to_csv("./data/all_tables_Spain.csv",index=None)

In [224]:
def clean_data(x):
    if isinstance(x,float):
        return x
    if "w." in x:
        return np.NaN
    number = x.split("/")[0].split("?")[0].split("–")[0].split("[")[0].strip()
    if len(number) == 0:
        return np.NaN
    if "<" in number:
        return np.NaN
    try:
        return float(number)
    except:
        print(number)
        print(len(number))
        

In [235]:
#clean data
spain = pd.read_csv("data/all_tables_Spain.csv",parse_dates=["date"])
for col in spain.columns:
    if col.lower() in types:
        continue
    if "Unnamed" in col:
        continue
    spain[col] = spain[col].apply(clean_data).astype(float)
    

In [236]:
#Combining parties
spain["PSOE"] = spain[["PSOE",'/wiki/Galician_Socialist_Party%E2%80%93Galician_Left','/wiki/PSM%E2%80%93Nationalist_Agreement']].sum(1)
spain["BNG"] = spain[["BNG–Nós","/wiki/Galician_Nationalist_Bloc"]].sum(1)
spain["IU"] = spain[["IU","IU–LV"]].sum(1)
spain["PP"] = spain[["PP","AP","AP-PDP-PL","AP–PDP"]].sum(1)
spain["Podemos"] = spain[["IU–UPeC","Podemos","Isotipo Unidos Podemos.svg"]].sum(1)

parties = set(list(spain.columns))

#Basic cleaning and augmentating
spain["firm"] = spain["firm"].str.split("[").str[0]
spain = spain.loc[~spain["firm"].apply(lambda x: "election" in x)] #results of the elections, not polls
spain = spain.loc[~spain["firm"].apply(lambda x: "^" in x)] #weird polls
spain["Party_poll"] = spain["firm"].apply(lambda x: any([p in x for p in parties]))
spain["Newspaper_poll"] = spain["firm"].apply(lambda x: x.split("/")[-1] if len(x.split("/"))>1 else np.NaN)
spain["Polling_firm"] = spain["firm"].str.split("/").str[0]
spain["Number_polling_firms"] = spain["firm"].str.count('–')+1

del spain["Unnamed: 0"]
spain = spain.replace(0,np.NaN)
#spain = spain.dropna(thresh=len(spain)/5,axis=1)
spain.to_csv("./data/all_tables_Spain_cleaned.csv")

In [237]:
spain = spain.dropna(thresh=len(spain)/5,axis=1)
spain.tail()

,CC,CiU,ERC,IU,PNV,PP,PSOE,Podemos,UPyD,date,firm,lead,size,turnout,Party_poll,Newspaper_poll,Polling_firm,Number_polling_firms
1975,0.62,3.012,NaN,NaN,1.15,33.5144,23.289,19.246,4.990,2014-07-05,Celeste-Tel/eldiario.es,10.3,1100.0,58.8,False,eldiario.es,Celeste-Tel,1
1976,NaN,2.400,NaN,NaN,1.60,28.8000,14.700,32.700,6.400,2014-06-14,Simple Lógica,2.1,1035.0,NaN,False,NaN,Simple Lógica,1
1977,NaN,NaN,NaN,NaN,NaN,34.6000,21.900,22.400,5.300,2014-06-11,NC Report,12.7,955.0,NaN,True,NaN,NC Report,1
1978,0.62,3.012,NaN,NaN,1.16,32.4139,22.394,19.541,5.712,2014-06-06,Celeste-Tel/eldiario.es,10.1,1100.0,57.9,False,eldiario.es,Celeste-Tel,1
1979,NaN,3.412,NaN,NaN,NaN,25.6125,19.487,23.271,7.813,2014-05-31,GESOP/El Periódico,6.2,1000.0,NaN,False,El Periódico,GESOP,1


# Systematical

In [ ]:
#Opinion polling by country
https://en.wikipedia.org/wiki/Category:Opinion_polling_by_country